Libraries

In [29]:
import os
import cv2
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from PIL import Image
from sklearn.metrics import classification_report
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer

Constants

In [30]:
IMAGE_SIZE = 64  

Pre Processing

In [26]:
execution_start_time = time.time()
data = []
labels = []

# Function to prepare the dataset
def prepareDataset(path):
    filePaths = []
    y = []
    count = 0
    name = ''

    for dirname, _, filenames in os.walk(path):
        if count != 0:
            x = dirname.split('/')[-1]
            index = x.rindex('_name_')
            name = x[index+6:].replace(' ','')
        for filename in filenames:
            full_path = os.path.join(dirname, filename)
            filePaths.append(full_path)
            y.append(name)
        
        count += 1
    return filePaths, y

hispanic_path = 'ID_Images_Dataset/Selfies_ID_Images_dataset/11_sets_Hispanics'  
caucasian_path = 'ID_Images_Dataset/Selfies_ID_Images_dataset/18_sets_Caucasians'  

hispanic_filePaths, hispanic_labels = prepareDataset(hispanic_path)
caucasian_filePaths, caucasian_labels = prepareDataset(caucasian_path)


filePaths = hispanic_filePaths + caucasian_filePaths
y_labels = hispanic_labels + caucasian_labels  

# Load and preprocess the images
for i, filePath in enumerate(filePaths):
    if not filePath.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"Skipping non-image file: {filePath}")
        continue
    try:
        pil_image = Image.open(filePath).convert('RGB')
        image = np.array(pil_image)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        data.append(image)
        labels.append(y_labels[i])
    except (IOError, OSError) as e:
        print(f"Warning: Could not read image from {filePath} - {e}")
#
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# Convert the labels from strings to integers
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Define the number of classes for the CNN model
number_of_classes = labels.shape[1]
print("Classes Count: ",number_of_classes)
np.save('label_classes.npy', lb.classes_)

Skipping non-image file: ID_Images_Dataset/Selfies_ID_Images_dataset/11_sets_Hispanics/.DS_Store
Skipping non-image file: ID_Images_Dataset/Selfies_ID_Images_dataset/18_sets_Caucasians/.DS_Store
Classes Count:  29


Model

In [24]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(number_of_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4), 
              metrics=['accuracy'])

start_time = time.time()
model.fit(data, labels, epochs=50, batch_size=32)
training_time = time.time() - start_time
print(f"Total training time: {training_time:.2f} seconds")
model.save('face_recognition_model.h5')

Epoch 1/50
14/14 [==============================] - 1s 53ms/step - loss: 3.3984 - accuracy: 0.0322
Epoch 2/50
14/14 [==============================] - 1s 51ms/step - loss: 3.3679 - accuracy: 0.0391
Epoch 3/50
14/14 [==============================] - 1s 51ms/step - loss: 3.3387 - accuracy: 0.0506
Epoch 4/50
14/14 [==============================] - 1s 56ms/step - loss: 3.3062 - accuracy: 0.0713
Epoch 5/50
14/14 [==============================] - 1s 53ms/step - loss: 3.2933 - accuracy: 0.0736
Epoch 6/50
14/14 [==============================] - 1s 52ms/step - loss: 3.2527 - accuracy: 0.1149
Epoch 7/50
14/14 [==============================] - 1s 50ms/step - loss: 3.1845 - accuracy: 0.1264
Epoch 8/50
14/14 [==============================] - 1s 53ms/step - loss: 3.1838 - accuracy: 0.1195
Epoch 9/50
14/14 [==============================] - 1s 50ms/step - loss: 3.0725 - accuracy: 0.1609
Epoch 10/50
14/14 [==============================] - 1s 50ms/step - loss: 2.9407 - accuracy: 0.1862
Epoch 11/

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Classification Report

In [28]:
model = load_model('face_recognition_model.h5')
predictions = model.predict(data)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(labels, axis=1)
lb.classes_ = np.load('label_classes.npy')
report = classification_report(true_classes, predicted_classes, target_names=lb.classes_)
print(report)
end_execution_time = time.time() - execution_start_time
print(f"Total Execution Time: {end_execution_time} seconds")

14/14 [==============================] - 0s 14ms/step
                 precision    recall  f1-score   support

      Alejandra       1.00      1.00      1.00        15
     Alessandro       1.00      1.00      1.00        15
      Anastasia       1.00      1.00      1.00        15
      AndreaRan       1.00      1.00      1.00        15
           Anna       1.00      1.00      1.00        15
          Bruno       1.00      1.00      1.00        15
       Clarissa       1.00      1.00      1.00        15
         Daiane       1.00      1.00      1.00        15
          Diego       1.00      1.00      1.00        15
            Ewa       1.00      1.00      1.00        15
       Fernanda       1.00      1.00      1.00        15
        Gabriel       1.00      0.93      0.97        15
GültenÇayırcı       0.94      1.00      0.97        15
        Juliana       1.00      0.93      0.97        15
          Kasia       1.00      0.93      0.97        15
       Kateryna       1.00      1